<a href="https://colab.research.google.com/github/gkunal8019/Find_your_doccumnet_by_llm/blob/main/Find_your_do_doccumnet_by_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/chat_engine/chat_engine_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Chat Engine - Context Mode

ContextChatEngine is a simple chat mode built on top of a retriever over your data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


For each chat interaction:
* first retrieve text from the index using the user message
* set the retrieved text as context in the system prompt
* return an answer to the user message

This approach is simple, and works for questions directly related to the knowledge base and general interactions.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nlt

In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.1 MB/s eta 0:00:00


## Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-10-01 06:55:56--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.008s  

2024-10-01 06:55:56 (9.04 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



## Get started in 5 lines of code

Load data and build index

In [ ]:
import openai
import os

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
!pwd

/content


In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Define the directory containing your Markdown files
input_dir = "./drive/MyDrive/ccs_computer/icmr_parsed/"

# Initialize a list to hold the data
all_data = []

# Loop through each subdirectory and file in the specified directory
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.md'):  # Check if the file is a Markdown file
            file_path = os.path.join(root, file)  # Get the full path of the file

            try:
                # Read the data from the Markdown file
                reader = SimpleDirectoryReader(input_dir=file_path)
                data = reader.load_data()
                all_data.extend(data)  # Add the loaded data to the list
            except ValueError as e:
                print(f"Error: {e}. Skipping {file_path}.")
            except Exception as e:
                print(f"An unexpected error occurred: {e}. Skipping {file_path}.")

# Create the index from all collected data
index = VectorStoreIndex.from_documents(all_data)

# Optional: Display the number of documents loaded
print(f"Loaded {len(all_data)} documents into the index.")

Error: Directory ./drive/MyDrive/ccs_computer/icmr_parsed/Vacancy_Circular_0001_13092024/Vacancy_Circular_0001_13092024.md does not exist.. Skipping ./drive/MyDrive/ccs_computer/icmr_parsed/Vacancy_Circular_0001_13092024/Vacancy_Circular_0001_13092024.md.
Error: Directory ./drive/MyDrive/ccs_computer/icmr_parsed/OfficeMemorandum_0001/OfficeMemorandum_0001.md does not exist.. Skipping ./drive/MyDrive/ccs_computer/icmr_parsed/OfficeMemorandum_0001/OfficeMemorandum_0001.md.
Error: Directory ./drive/MyDrive/ccs_computer/icmr_parsed/Memorandum/Memorandum.md does not exist.. Skipping ./drive/MyDrive/ccs_computer/icmr_parsed/Memorandum/Memorandum.md.
Error: Directory ./drive/MyDrive/ccs_computer/icmr_parsed/Invite_for_expre_interest-reg/Invite_for_expre_interest-reg.md does not exist.. Skipping ./drive/MyDrive/ccs_computer/icmr_parsed/Invite_for_expre_interest-reg/Invite_for_expre_interest-reg.md.
Error: Directory ./drive/MyDrive/ccs_computer/icmr_parsed/icmr/icmr.md does not exist.. Skipping

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
data = SimpleDirectoryReader(
    input_dir="./drive/MyDrive/ccs_computer/icmr_parsed",
    required_exts=['.md'],
    recursive=True,
).load_data()
index = VectorStoreIndex.from_documents(data)

In [ ]:
reader.load_data()

[Document(id_='b6491802-62eb-449f-a3f7-3278880d762a', embedding=None, metadata={'file_path': '/content/drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_0/General_Circular_0001_0.md', 'file_name': 'General_Circular_0001_0.md', 'file_type': 'text/markdown', 'file_size': 45461, 'creation_date': '2024-10-01', 'last_modified_date': '2024-10-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='BN / GRAM\nSCIENTIFIC\nww.icmr.nic.in implication of sansad, nic, in 011-26588662-011-26589791-011-265 E mail भारतीय आयुविज्ञान अनुसंधान परिषद INDIAN COUNCIL OF MEDICAL RESEARCH\nरवारथ्य अनुराधान थिमाग ( स्वास्थ्य एव परिधार ) कल्पाण मंत्रालय)\nth रामलिगर्चामी भयन, अन्सारी नगर, नई : दिल्ली - 110029 DEPARTMENT OF HEALTH RESEARCH (MINISTRY  OF HEALTH & FAMILY W

In [ ]:


# data = SimpleDirectoryReader(input_dir="./drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_0").load_data()
# index = VectorStoreIndex.from_documents(data)

Configure chat engine

Since the context retrieved can take up a large amount of the available LLM context, let's ensure we configure a smaller limit to the chat history!

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
    ),show_source=True,
)

Chat with your data

In [ ]:
# Assuming the chat engine is already set up

# Run a query through the chat engine
response = chat_engine.chat("what is icmr")

# # Access the main response text
# print("Response:", response.response)

# # If sources are available, print them
# if hasattr(response, 'source_documents') and response.source_documents:
#     print("\nSources:")
#     for doc in response.source_documents:
#         print(f"Document: {doc.metadata['source']}")  # or access metadata as appropriate
# else:
#     print("No sources found.")


In [ ]:
response.sources

[ToolOutput(content="[NodeWithScore(node=TextNode(id_='a8288264-e443-4bd1-966b-0568267b2b0a', embedding=None, metadata={'file_path': '/content/drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_1/General_Circular_0001_1.md', 'file_name': 'General_Circular_0001_1.md', 'file_type': 'text/markdown', 'file_size': 34395, 'creation_date': '2024-10-01', 'last_modified_date': '2024-10-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='35724cfd-089e-4c3d-a5ec-0d39626b32a7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_1/General_Circular_0001_1.md', 'file_name': 'General_Circular_0001_1.md', 'file_type': 'text/markdown

In [ ]:
kunal=response.sources[0].content

In [ ]:
response.sources[0].content

"[NodeWithScore(node=TextNode(id_='a8288264-e443-4bd1-966b-0568267b2b0a', embedding=None, metadata={'file_path': '/content/drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_1/General_Circular_0001_1.md', 'file_name': 'General_Circular_0001_1.md', 'file_type': 'text/markdown', 'file_size': 34395, 'creation_date': '2024-10-01', 'last_modified_date': '2024-10-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='35724cfd-089e-4c3d-a5ec-0d39626b32a7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/ccs_computer/icmr_parsed/General_Circular_0001_1/General_Circular_0001_1.md', 'file_name': 'General_Circular_0001_1.md', 'file_type': 'text/markdown', 'file_size': 3439

In [ ]:
import re

# Sample input string (the output you provided)

# Define a regex pattern to capture id_, text, and score
pattern = r"id_='(.*?)',.*?text='(.*?)',.*?score=(\d+\.\d+)"

# Find all matches in the input string
matches = re.findall(pattern, kunal)

# Create a list of dictionaries to store the extracted data
results = [{'id_': match[0], 'text': match[1], 'score': float(match[2])} for match in matches]

# Print the results
for result in results:
    print(result)

{'id_': 'a8288264-e443-4bd1-966b-0568267b2b0a', 'text': 'निदेशक/प्रभारी  निदेशक परिषद के सभी संस्थान/केन्द्र स स  में. कमांक.E16/68/2022-प्रशासन/E.Office.-142748 दिनाक:19/09/2024 स्वास्थ्य अनुसधान विभाग ( स्वास्थ्य एव परिवार कल्याण मत्तालय)\\nवी रामलिंगस्वामी भवन, अन्सारी, नगर, नई (दिल्ली - 110029)\\nDEPARTMENT OF HEALTH RESEARCH (MINISTRY OF HEALTH & FAMILY WELFARE)\\nV. RAMALINGASWAMI BHAWAN, ANSARI NAGAR, NEW DELHI-110029 भारतीय आयुविज्ञान अनुसंधान परिषद INDIAN COUNCIL OF MEDICAL RESEARCH\\n4.4.4.4228./PABX : 26588980, 26588707, 26589336, 26589745 26589873, 26589414 011-26588662, 011-26589791, 011-26589258 तार / GRAM\\nWebsite to the www.icmr.nic.in E-mail 5 immune to sansad nic, in', 'score': 0.7806064497011477}
{'id_': '9adc3bdc-f238-44df-aea5-a420ea2a203c', 'text': 'एम.एस  और  ई-मेलों  का  भी  तुरंत और वरीयता के आधार पर जवाब दिया जाना चाहिए ;\\n(xiii)     सभी   मंत्रालय/विभाग  यह   सुनिश्चित  कर्रे   कि   केन्द्र    सरकार   द्वारा   प्रायोजित   विभिन्न सरकारी    योजनाओं    के    

In [ ]:
pattern = r"id_='(.*?)',.*?text='(.*?)',.*?score=(\d+\.\d+)"

# Find all matches in the input string
matches = re.findall(pattern, kunal)

# Initialize result as an empty list
result = []

# Append each extracted match to the result list
for match in matches:
    result.append({'id_': match[0], 'text': match[1], 'score': float(match[2])})

# Now result contains all values
print(result)

[{'id_': 'a8288264-e443-4bd1-966b-0568267b2b0a', 'text': 'निदेशक/प्रभारी  निदेशक परिषद के सभी संस्थान/केन्द्र स स  में. कमांक.E16/68/2022-प्रशासन/E.Office.-142748 दिनाक:19/09/2024 स्वास्थ्य अनुसधान विभाग ( स्वास्थ्य एव परिवार कल्याण मत्तालय)\\nवी रामलिंगस्वामी भवन, अन्सारी, नगर, नई (दिल्ली - 110029)\\nDEPARTMENT OF HEALTH RESEARCH (MINISTRY OF HEALTH & FAMILY WELFARE)\\nV. RAMALINGASWAMI BHAWAN, ANSARI NAGAR, NEW DELHI-110029 भारतीय आयुविज्ञान अनुसंधान परिषद INDIAN COUNCIL OF MEDICAL RESEARCH\\n4.4.4.4228./PABX : 26588980, 26588707, 26589336, 26589745 26589873, 26589414 011-26588662, 011-26589791, 011-26589258 तार / GRAM\\nWebsite to the www.icmr.nic.in E-mail 5 immune to sansad nic, in', 'score': 0.7806064497011477}, {'id_': '9adc3bdc-f238-44df-aea5-a420ea2a203c', 'text': 'एम.एस  और  ई-मेलों  का  भी  तुरंत और वरीयता के आधार पर जवाब दिया जाना चाहिए ;\\n(xiii)     सभी   मंत्रालय/विभाग  यह   सुनिश्चित  कर्रे   कि   केन्द्र    सरकार   द्वारा   प्रायोजित   विभिन्न सरकारी    योजनाओं    के  

In [ ]:
result # This will print each result on a new line

[{'id_': 'a8288264-e443-4bd1-966b-0568267b2b0a',
  'text': 'निदेशक/प्रभारी  निदेशक परिषद के सभी संस्थान/केन्द्र स स  में. कमांक.E16/68/2022-प्रशासन/E.Office.-142748 दिनाक:19/09/2024 स्वास्थ्य अनुसधान विभाग ( स्वास्थ्य एव परिवार कल्याण मत्तालय)\\nवी रामलिंगस्वामी भवन, अन्सारी, नगर, नई (दिल्ली - 110029)\\nDEPARTMENT OF HEALTH RESEARCH (MINISTRY OF HEALTH & FAMILY WELFARE)\\nV. RAMALINGASWAMI BHAWAN, ANSARI NAGAR, NEW DELHI-110029 भारतीय आयुविज्ञान अनुसंधान परिषद INDIAN COUNCIL OF MEDICAL RESEARCH\\n4.4.4.4228./PABX : 26588980, 26588707, 26589336, 26589745 26589873, 26589414 011-26588662, 011-26589791, 011-26589258 तार / GRAM\\nWebsite to the www.icmr.nic.in E-mail 5 immune to sansad nic, in',
  'score': 0.7806064497011477},
 {'id_': '9adc3bdc-f238-44df-aea5-a420ea2a203c',
  'text': 'एम.एस  और  ई-मेलों  का  भी  तुरंत और वरीयता के आधार पर जवाब दिया जाना चाहिए ;\\n(xiii)     सभी   मंत्रालय/विभाग  यह   सुनिश्चित  कर्रे   कि   केन्द्र    सरकार   द्वारा   प्रायोजित   विभिन्न सरकारी    योजनाओं 

In [ ]:
hight_light_text=result[0]['text'][:100]

In [ ]:
hight_light_text

'निदेशक/प्रभारी  निदेशक परिषद के सभी संस्थान/केन्द्र स स  में. कमांक.E16/68/2022-प्रशासन/E.Office.-14'

In [ ]:
import os

def search_and_highlight_and_save_md_files(base_directory, search_text, output_directory):
    """
    Search for text in all .md files within the specified directory and its subdirectories,
    highlight the matching text, and save the modified files to a new directory with the same structure.

    Parameters:
    base_directory (str): The path of the directory to search in.
    search_text (str): The text to search for within the .md files.
    output_directory (str): The path of the directory to save the modified files.

    Returns:
    None
    """
    # Check if the base directory exists
    if not os.path.exists(base_directory):
        print(f"Directory {base_directory} does not exist.")
        return

    # Walk through all subdirectories and files
    for dirpath, _, filenames in os.walk(base_directory):
        for filename in filenames:
            if filename.endswith('.md'):
                file_path = os.path.join(dirpath, filename)
                try:
                    # Read the contents of the file
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()

                    # Check if the search text exists in the content
                    if search_text in content:
                        # Highlight the matching text by wrapping it with brackets
                        highlighted_text = f"$$\\textcolor{{red}}{{{search_text}}}$$"
                        highlighted_content = content.replace(search_text, highlighted_text)

                        # Determine the new file path in the output directory
                        relative_path = os.path.relpath(file_path, base_directory)
                        new_file_path = os.path.join(output_directory, relative_path)

                        # Create the new directory structure if it doesn't exist
                        new_dir = os.path.dirname(new_file_path)
                        os.makedirs(new_dir, exist_ok=True)

                        # Write the modified content to the new file
                        with open(new_file_path, 'w', encoding='utf-8') as new_file:
                            new_file.write(highlighted_content)

                        print(f"Modified file saved: {new_file_path}")
                except Exception as e:
                    print(f"Could not process {file_path}: {e}")

    return new_file_path

# Example usage:
base_directory = '/content/drive/MyDrive/ccs_computer/icmr_parsed'       # Set the base directory where the .md files are located
search_text = hight_light_text                               # Set the text to search for
output_directory = 'icmi_highlight01'                                      # Set the directory to save the modified files

# Call the function
kunal = search_and_highlight_and_save_md_files(base_directory, search_text, output_directory)

Modified file saved: icmi_highlight01/General_Circular_0001_1/General_Circular_0001_1.md


In [ ]:
kunal

'icmi_highlight01/General_Circular_0001_1/General_Circular_0001_1.md'

In [ ]:
from IPython.display import Markdown, display

# Define the path to your Markdown file
markdown_file_path = kunal  # Change this to your file's path

# Read the Markdown file
with open(markdown_file_path, 'r') as file:
    markdown_content = file.read()

# Display the Markdown content
display(Markdown(markdown_content))

Signed by Jagdish Rajesh Date: 23-09-2024 16:44:2 S ग द ी श  रा ज श सस भवदीय,

| SI.No Reference No. & Date                                                | Name of Ministry                            | Subject                              |                                |                                           |
|---------------------------------------------------------------------------|---------------------------------------------|--------------------------------------|--------------------------------|-------------------------------------------|
| No. NIC-CSG/2024-08/502 Advisory and ResourceImportant Advisor:Protecting |                                             |                                      |                                |                                           |
| 1.                                                                        | Dated: 16.08.2024                           | Facilitation,                        | National Sensitive Information | d                                         |
| Informatics                                                               | Centre, Awareness                           | on                                   | Phishing                       |                                           |
| Lodhi Road New Delhi                                                      | Domains                                     |                                      |                                |                                           |
| 2.                                                                        | O.M. No. 31/22/2024-EODepartment            | of Policy on fixing an upper limit   |                                |                                           |
| Personnel & Training of the number of years for                           |                                             |                                      |                                |                                           |
| (MM-I)                                                                    |                                             |                                      |                                |                                           |
| Dated: 20.08.2024                                                         | (Office                                     | of                                   | the which officers can work in |                                           |
| Establishment Officer), the personal staff of the                         |                                             |                                      |                                |                                           |
| New Delhi                                                                 | Union Minister in their entire              |                                      |                                |                                           |
| careers-reg.                                                              |                                             |                                      |                                |                                           |
| 3.                                                                        | D.O.                                        | No.                                  | A- Department y                | of  Legal D.O. Letter Inviting Details of |
| 60011/19/2022-IIAC(Pt.-1) Affairs, Ministry of Law pending/closed         | Arbitration                                 |                                      |                                |                                           |
| Dated: 02.09.2024                                                         | & Justice, New Delhi                        | matters involving .                  | Uol                            | as                                        |
| party.                                                                    |                                             |                                      |                                |                                           |
| 4.                                                                        | D.O.                                        | No. Department of Health Celebration | of                             | National                                  |
| 01/10/NVBDD/DGHS/2024/and                                                 | Family                                      | Welfare, Volutary Blood Donation Day |                                |                                           |
| Dated: 12.09.2024                                                         | Ministry of Health and on 1 st October-reg. |                                      |                                |                                           |
| Family Welfare, Nirman                                                    |                                             |                                      |                                |                                           |
| Bhawan, New Delhi                                                         |                                             |                                      |                                |                                           |

विभिन्न मंतालयो/विभागो से प्रास निस्नलिखित अर्द्व सरकारी पव / कार्यालय जापन , सूचना एव आवश्यक कार्यवाही के लिए संलग्न है।
महोदय/महोदया .

$$\textcolor{red}{निदेशक/प्रभारी  निदेशक परिषद के सभी संस्थान/केन्द्र स स  में. कमांक.E16/68/2022-प्रशासन/E.Office.-14}$$2748 दिनाक:19/09/2024 स्वास्थ्य अनुसधान विभाग ( स्वास्थ्य एव परिवार कल्याण मत्तालय)
वी रामलिंगस्वामी भवन, अन्सारी, नगर, नई (दिल्ली - 110029)
DEPARTMENT OF HEALTH RESEARCH (MINISTRY OF HEALTH & FAMILY WELFARE)
V. RAMALINGASWAMI BHAWAN, ANSARI NAGAR, NEW DELHI-110029 भारतीय आयुविज्ञान अनुसंधान परिषद INDIAN COUNCIL OF MEDICAL RESEARCH
4.4.4.4228./PABX : 26588980, 26588707, 26589336, 26589745 26589873, 26589414 011-26588662, 011-26589791, 011-26589258 तार / GRAM
Website to the www.icmr.nic.in E-mail 5 immune to sansad nic, in

## पपपप

1. महानिदेशक/अपर महानिदेशक/वरि.उपमहानिदेशक (प्रशा.)/वरि.वित सलाहकार के निजी सचिय 2. परिषद के सभी प्रभाग प्रमुख 3. 4. प्रमुख वीएमआई - आईसीएमआर की वेवसाइट पर अपलोड - करने के अनुरो - के साथा
ADMN-II ADMN <admn2355@gmail.com Fwd: Fwd: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Domains R Lakshminarayanan <lakshminarayanan.r@icmr.gov.in> To: admn2355 <admn2355@gmail.com>
C: Jagdish Rajesh <rajeshj.nq@icmr.gov.in> , Saritha Nair <nairs@icmr.gov.in> , KARMARKAR TUSHAR <tushar.arun@gov.in>
Sun, Aug 18, 2024 at 10:18 A
Please circulate.

Thanks Dr R Lakshminarayanan Deputy Director General (A) &
Vigilance Officer ICMR HQ, New Delhi.

---- Forwarded Message ---
From: KARMARKAR TUSHAR <tushar.arun@gov.in>
To: Saritha Nair <nairs@icmr.gov.in>, R Lakshminarayanan <lakshminarayanan.r@icmr.gov.in>, dhrhq.icmr.<dhrhq.iomr@gnail.com>, MD SOHAIL AHMAD KHAN <sohall.akhan@ionr.gov.in>, giadhr21@gmail.com, Roopa Hariprasad <roopa.hariprasad@gov.in>, Vikas Dhikav <dhikav.v@iomr.gov.in>
Cc: Shiv Kumar <shiv.kumar1995@gov.in>
Sent: Sun, 18 Aug 2024 07:38:34 +0530 (IST)
Subject: Fwd: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Domains Dear all, Kindly circulate among your section/vertical staff Regards Dr Tushar Karmarkar Deputy Secretary, Department of Health Research From: "Shiv Kumar" <shiv.kumar1995@gov.in>
To: "KARMARKAR TUSHAR" <tushar arun@gov.in>, "Vijay Kumar" <vijay.kumar68@nic.in>, "Rajeev.Saxena" <saxena.rajeev@n.c.in>, "Kavitha Rajsekar" < kavitha.rajsekar@nic.in>, dhrpmuvdi2@gmail.com, "Sujata Sinha" < Sujata.sinha@nic.in >
C: "Biswabandan Senapati" <b.senapati@nic.in>, "JACOB LALMALSAWM" <jacob.shakum@nic.in>
Sent: Friday, August 16, 2024 5:22:08 PM
Subject: Fwd: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Domains Sir/Madam, Kindly find attached the advisory for 'Phishing Domains mimicking Central Bureau of Investigation'. It is requested to circulate the same in your division/sections.

Thanks & Regards Shiv Kumar Assistant Section Officer Department of Health Research Ministry of Health & Family Welfare From: "Biswabandan Senapati" <b.senapati@nic.in>
To: "Shiv Kumar" <SHIV.KUMAR1995@GOV.IN> Cc: "N Sriramasubramanian" <sreram-law@nic.in>
Sent: Friday, August 16, 2024 5:11:29 PM
Subject: Fwd: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Domains B.Senapati, Director D/o Health Research M/o Health and Family Welfare Tel.: 2373 6090 Mob : +91 11 9971229301 From: "CSG Advisory Team" <csg-advisory@nic.m>
To: "ciso-list" <ciso-list@lsmgr.nic.in>
C: "Anupam Srivastava" <hod-art@nic.in>, "seema khanna" <hod-es-eg@nic.in>, "hod-list" <hod-list" <hod-list@isnig.in>, "hog-list" <hod-list@isnig.in>,
https://mail.google.com/mail/u/0/?ik=41c48f63f8&view=pt&search=all&permmsgid=msg-f:1807699421503900537&simpl=msg-f:1807699421503900537 1/
8/20/24, 10:21 AM
Gmail - Fwd: Fwd: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Doriains
"dciso-list" <dciso-list@ismgr.nic.in>, "Shilpi Jain" <officer2.art@nic.in>, "vikram dhull" <officer5.art@nic.in>, "shray gahallot" <officer4.art@nic.

Sent: Friday, August 16, 2024 5:00:53 PM
Subject: [Ciso-list] Important Advisory: Protecting Sensitive Information & Awareness on Phishing Domains This mail is being sent with the Approval of HOG (Cyber Security Group) Dr. Ms. Seema Khanna Dear Sir/Ma'am, Kindly refer the attached advisory for 'Phishing Domains mimicking Central Bureau of Investigation'.

The said phishing campaign is primarily aimed to harvest confidential documents of Indian Citizens like their Aadhar Cards, PAN cards and to gather documents of Indian Citizens like their Aadhar Cards, PAN cards and to gather their financial details to carry out malic ous activities in Indian Cyber Space. In this regard, you are requested to circulate the attached advisory to all officials in your ministry/department.

Regards CIS Governance Division Cyber & Information Security Group
[ https://amritmahotsav.nic.in/ ]
Disclaimer: This e-mail and its attachments may contain official Indian Government information. If you are not the intended recipient, please notify the sender immediately and delete this e-mail. Any dissemination or use of this information by a person other than the intended recipient is unauthorized. The responsibility lies with the recipient to check this email and any attachment for the presence of viruses.

Ciso-list mailing list - ciso-list@lsmgr.nic.in To unsubscribe send an email to ciso-list-leave@lsmgr.nic.in 2 attachments 502 - Advisory for Phishing Domain Mimicking CBI.pdf 200K Biswabandan Senapati.vcf 1 1K
https://mail.google.com/mail/u/0/?ik=41c48f63f8&view=pt&search=all&permmsgid=msg-f:1807699421503900537&simpl=msg-f:1807699421503900537 vivisory and Resource tacilitation

Venoral antormality Contr PN0557/3 ABILITY AND ECOMPLEX Reduction New Delai - 11000K India ese advisory@nic.h

## Advisory For Phishing Domains Mimicking Central Bureau Of Investigation Description:

During cyber investigation below mentioned 2 phishing domains are found mimicking Central Bureau of Investigation (CBI) under Government of India. The phishing campaign is primarily aimed to harvest confidential documents of Indian Citizens like their Aadhar Cards, PAN cards and to gather documents of Indian Citizens like their Aadhar Cards, PAN cards and to gather their financial details to carry out malicious activities in Indian Cyber Space.

1.

2.

cbigovins.top/app-in2/
cbigovins.site/app-in2/
It is pertinent to mention that there is an "Upload Aadhar Card (JPG)" button which is collecting Front and Back Images of the Aadhar Card and the PAN Card. There is also a "Self-Declaration Form" which is collecting financial details such as Bank Name, Account Number, Mobile banking user id, Credit/Debit card details etc. of Indian citizens so as to carry out malicious activities in the Indian Cyber Space.

## In View Of Above, Nic-Cyber Security Group Advises Following:

1. In case such a phishing mail is received, do not enter or upload your any personal information/document, when redirected to any upload/login page.

2. Delete these phishing emails from your inbox.

3. In case, you have already clicked the phishing URL and uploaded/submitted your confidential information:
a.  Change your internet banking/mobile banking password - You need to change the passwords for any accounts that might have been hit in the cyberattack.

b. Turn on multi-factor authentication for the account that might have been attacked.

 Review your bank account and credit card statements on a regular basis C.

d.   Make sure there are no unauthorized transactions Report any unauthorized transactions immediately e.

Block your credit/debit card - Your cards can be misused by hackers even if they do f.

nt know the PIN.

By following these steps you can effectively mitigate the potential risks associated with clicking on the above-mentioned phishing URL.

Disconnect from the Internet.

a.

b. Scan your device with anti-virus software.

Enable Two-Factor (2FA) Authentication on all critical accounts to add an extra layer c.

of security.

d. Review and Update Security Settings - review the security settings on your digital accounts and devices and ensure that your software and applications are up to date to defend against new threats.

Back up your files.

e.

Report it to your security team.

f.

## Some Ways To Recognise A Phishing Email, Are Given Below:

Be suspicious of emails that claim you must click, call, or open an attachment a.

immediately or urgently.

b. If a mail received from unknown source, this may be a source of phishing.

If an email message has obvious spelling or grammatical errors, it might be a scam.

C...

E.g. nlc.in where the first "i" has been replaced by "l", or gov.in, where the "o" has been replaced by a "0" (zero).

d. Images of text used in place of text (in messages or on linked web pages) may be scam.

Be cautious of links shortened by using Bit.Ly or other link shortening techniques.

e.

DG. ICM
DG ICMR
[Secy-goi] Policy on fixing an upper limit of the number of years for which officers can work in the personal staff of the union Minister in their entire careers-reg.

From : Praveen <p.sundriyal@gov.in>
Tue, Aug 20, 2024 06:38 PM
 1 attachment Subject : [Secy-goi] Policy on fixing an upper limit of the number of years for which officers can work in the personal staff of the union Minister in their entire creers-reg.

To : Shri Rajat Kumar ,IAS,CG,2005 Chattisgarh
<rajat.kr@ias.nic.in>, secy-goi <secygoi@Ismgr.nic.in>
Cc : RUPESH KUMAR SECTION OFFICER (MM-I) DOPT
<.rupesh@nic.in>
Sir/Madam, Please find attached the subject item for kind perusal/action as deemed appropriate Regards Praveen Section Officer DOPT
Secy-goi mailing list -- secy-goi@lsmgr.nic.in To unsubscribe send an email to secy-goi-leave@lsmgn.nic.in relaxation of upper limit of 10 years.pdf 198 KB
mail No. 31/22/2024-EO (MM-I)
Government of India Department of Personnel & Training (Office of the Establishment Officer)
North Block, New Delhi Dated: 20 August, 2024.

## Office Memorandum

Policy on fixing an upper limit of the number of years for which officers can Subject :
work in the personal staff of the Union Minister in their entire careers.

Reference is invited to DoP&Ts O.M. No. 31/65/2009-EO(MM-I) dated 04.3.2010 and O.M. No. 31/11/2010-EO(MM-I)  dated 13.05.2010 on the above subject.

In the above context, references are being received from Ministries / Departments 2.

seeking relaxation to DoPTs O.M. dated 04.03.2010 and 13.05.2010 in respect of the officials of Level 8 and below and of equivalent pay level for their appointment in personal staff of Minister.

It is clarified that the ceiling of tenure of 10 years of working in personal staff of Ministers 3.

as prescribed vide OMs dated 04.03.2010 and 13.05.2010 is not applicable to officers/officials of level 8 and below and of equivalent pay level and in such case, the concerned Ministries/Departments may take a decision as per extant guidelines.

(Sakshi Mittal)
Director
To, All Secretaries to the Government of India.

No. 31/22/2024-EO (MM-I)
Dated: 20 August, 2024.

Copy forwarded for information & necessary action to:- 1.

DP&T (Shri Rajat Kumar, Joint Secretary) CS-II Division, New Delhi.

2.

Prime Minister's Office (Ms. Manmeet Kaur, Deputy Secretary), New Delhi.

3.

4.

Cabinet Secretariat (Ms. Mrunmai Joshi, Deputy Secretary).

PS to MOS (PP)/Sr.PPS to Secretary (P).

5.

PS to EO/Dir.(MM)/RO(CM)/Guard File.

6.

Director, NIC for placing on the website of DoP&T.

(Sakshi Mittal)
Director
Email Fwd: pR: 3845088 - D.O. Letter Inviting Details of pending/closed Arbitration matters involving UoI as party .

From : R Lakshminarayanan
<lakshminarayanan.r@icmr.gov.in>
Thu, Sep 12, 2024 02:19 PM
 1 attachment Subject : Fwd: pR: 3845088 - D.O. Letter Inviting Details of pending/closed Arbitration matters involving UoI as party .

To : Administration ICMR <admn-hq@icmr.gov.in>
CC : admn2355@gmail.com, SAFAL CHETRI CHETRI
<chetri.s@icmr.gov.in>, Jagdish Rajesh
<rajeshj.hq@icmr.gov.in>, dhrhq icmr
<dhrhq.icmr@gmail.com>
Please circulate. Thanks Dr. R. Lakshminarayanan Deputy Director General (A) &
Vigilance Officer, ICMR HQ, New Delhi.

==
From: <dhrhq.icmr@gmail.com>
To: "R Lakshminarayanan"<lakshminarayanan.r@icmr.gov.in>
Cc: "MANISHA SAXENA"<srddga.hq@icmr.gov.in>, "Jagdish Rajesh"
<rajeshj.hq@icmr.gov.in>, <cbaisya@gmail.com>, "KARMARKAR TUSHAR"
<tushar.arun@gov.in>, "Jatin Singh"<jatin.singh90@gov.in>
Date: Thu, 12 Sep 2024 12:47:46 +0530 Subject: pR: 3845088 - D.O. Letter Inviting Details of pending/closed Arbitration matters involving UoI as party . == Sir, Kindly find attached a scanned copy of DO letter dated 02.09.2024 from Secretary, Department of Legal Affairs alongwith the format with a request to provide information regarding pending Arbitration Matters as per the proforma attached. The information may please be sent on priority.

With regards, DHR-ICMR Section.

.0. leading details of pending closed Aronratio...

nttps://mail.mgoveloud.m/zm/?fromService=wp&wpVersion=29ad
[Seey-goi] D.O. letter Inviting details of pending/closed Arbitration Matters involv Uol as Party Dr. Rajiv Mani <secylaw-dla@nic.in>
DR
Tue, 03 Sep 2024 10:18:06 AM +0530 To  "secretaries GOI" <secy-goi@lsmgr.nic.in>
Sir, Please find attached a D.O. letter from Dr. Rajiv Mani, Secretary, Department of Legal Affairs on the subject mentioned above. With regards,

Office of Secretary Department of Legal Affairs Ministry of Law & Justice 4th Floor, Shastri Bhawan New Delhi - 110 001 Tel. 011-23384205, 23387908
i -
s 1.

[Secy-goi] D.O. letter Inviting details of pending/closed Arbitratio...

https://mail.mgovcloud.jn/zm/?fromService=wp&wj.

Secy-goi mailing list -- secy-goi@lsmgr.nic.in To unsubscribe send an email to secy-goi-leave@lsmgr.nic.in 1. Attachment(s) -  Download as Zip 9 D.O Letter inviting details of pen... .pdf 511.9 KB - 240 विधि  कायं  विभाग विधि आंर न्याय मंत्रालय आ आरकाa DEPARTMENT OF LEGAL AFFAIRS
MINISTRY OF LAW & JUSTICE
GOVERNMENT OF INDIA
D.O.No.A-60011/19/2022-IIAC(Pt.-1)
September 2, 2024.

As you are aware, the Arbitration and Conciliation Act, 1996 (the Act) is the governing law relating to arbitrations in India and to keep pace with the current developments in the arbitration landscape and to enable arbitration as a viable dispute resolution mechanism, the indian arbitration law has undergone significant changes in the years 2015, 2019, and 2021.

The Department of Legal Affairs is in the process of considering further ci amendments in the Act to inter-alia ensure further strengthening of the arbitration ecosystem in the country. That apart, as you are aware, the Government of India is also a party to various arbitration proceedings and the subsequent related court litigations. To have a holistic overview of the arbitration cases and related court litigation, for considering legislative and policy interventions/amendments relating to arbitration, it has been desired that the details of pending/ closed Arbitration Matters involving Union of India as Party, are required to be examined.

3.

Therefore, I would like to request you to kindly provide the details of pending/
closed Arbitration Matters involving Union of India as Party since the year 2000, in the prescribed format (MS-Excel version), which is enclosed as Annexure-I, latest by 17th September, 2024 at ndiac-dlala.gov.in.

With regards.

Yours sincerely.

Encl: As above.

(Dr. Rajiv Mani)
To All Secretaries to the Government of India
(As per standard list)
Annexure-I

| |Details of pending Arbitration Matters involving Union of India as Party           |                                |                |
|-------------------------------------------------------------------------------------|--------------------------------|----------------|
| [ Total                                                                             | Name of                        | Fees paid Fees |
| Name of                                                                             | Date of                        | ISeat of       |
| Sr Case Value                                                                       | Itill 15                       | Structure      |
| Arbitration number of all                                                           |                                |                |
| No Title involved Arbitrator/slinitiation                                           | advocates Advocates Aug 2024   |                |
| (in                                                                                 | ot                             | engaged        |
| indian                                                                              | Arbitration                    | on behalf      |
| proceeding                                                                          |                                |                |
| (Rs.)                                                                               | of Union                       |                |
| of India                                                                            | If Appeal filed                |                |
| [Details of closed Arbitration Matters involving Union of India as Party since 2000 | Name of                        | lFees          |
| Court                                                                               | Section                        |                |
| iName of Fees paid Won/LostDate of                                                  |                                |                |
| Name of                                                                             | Date of                        | Total          |
| ISr. Case Value                                                                     | |34/37/SLP|Advocates|Structure |                |
| A A ward                                                                            | !Name                          |                |
| initation of number of [all                                                         |                                |                |
| Ine [Fitte]involved|Arbitrator                                                      | in Appeal                      |                |
| where                                                                               |                                |                |
| Arbitration/Advocates Advocates                                                     |                                |                |
| lin                                                                                 | I ' S                          | Appeal         |
| proceeding involved                                                                 |                                |                |
| indian                                                                              | is                             |                |
| lon behalf                                                                          |                                |                |
| IRs.                                                                                | pending                        |                |
| lof Union                                                                           |                                |                |
| of India                                                                            |                                |                |

9/13/24, 11:06 AM
Email Fwd: [Secy-goi] Celebration of National Voluntary Blood Donation Day on 1st October - reg.

From : R Lakshminarayanan
<lakshminarayanan.r@icmr.gov.in>
Thu, Sep 12, 2024 11:53 PM
 1 attachment Subject : Fwd: [Secy-goi] Celebration of National Voluntary Blood Donation Day on 1st October - reg.

To : Administration ICMR <admn-hq@icmr.gov.in>
CC : admn2355@gmail.com, SAFAL CHETRI CHETRI
<chetri.s@icmr.gov.in>, Jagdish Rajesh
<rajeshj.hg@icmr.gov.in>, DG ICMR <secydg@icmr.gov.in>
Please circulate.

Thanks Dr. R. Lakshminarayanan Deputy Director General (A) &
Vigilance Officer, ICMR HQ, New Delhi.

==
From: DG ICMR <secy-dg@icmr.gov.in>
To: "Anu Nagar"<anu.nagar1@gov.in>, "RICHA KHODA"<richa.khoda@gov.in>, "Manisha Saxena"<srddga.hq@icmr.gov.in>
Cc: "R Lakshminarayanan"<lakshminarayanan.r@icmr.gov.in>, "Jagdish Rajesh"
<rajeshj.hq@icmr.gov.in>, "Office DHR"<secy-dhr@gov.in>
Date: Thu, 12 Sep 2024 15:01:21 +0530 Subject: [Secy-goi] Celebration of National Voluntary Blood Donation Day on 1st October -
reg.

==
== From: Apurva Chandra <secyhfw@nic.in>
To: "all secretaries GoI secy"<secy-goi@lsmgr.nic.in>
Cc: "DGHS"<dghs@nic.in>, "mpkhobragade"<mp.khobragade@gov.in>
Date: Thu, 12 Sep 2024 14:38:20 +0530 Subject: [Secy-goi] Celebration of National Voluntary Blood Donation Day on 1st October -
reg.

==
Respected Sir/Madam, Please find attached a D.O. letter dated 12th September 2024 from Secretary, Department of Health and Family Welfare, Government of India on the above subject along with enclosures.

https://email.gov.in/n/printmessage?id=1289&tz=Asia/Kolkata&xim=1 apurva chandra, ias Secretary भारत  सरकार रवास्थ्य  एवं  परिवार  कल्याण  विभाग स्वास्थ्य, एवं  परिवार, कल्याण, मंत्रालय Government of India Department of Health and Family Welfare Ministry of Health and Family Welfare desigD.O.No.01/10/NVBDD/DGHS/2024 12 th September, 2024 Subject Celebration of National Voluntary Blood Donation Day
..

on 1s' of October.

Dese, Collectora National Voluntary Blood Donation Day on October 1st is a vital observance dedicated to acknowledging and celebrating the generosity of blood donors. By emphasizing the importance of voluntary, non-remunerated blood donation, the day seeks to inspire more individuals to contribute regularly. This act of giving not only saves lives but also ensures a continuous and safe supply of blood for those in need. The day also serves as a powerful reminder of the ongoing need for blood donations and the profound impact that each donation can have on the health and well-being of others.

The Campaign slogan for this year is "Celebrating 20 years of giving: Thank 2 you, blood donors!".

The objectives of this year's campaign are:
e To thank and recognize the millions of voluntary blood donors who have contributed to the health and well-being of millions of people around the world.

e To showcase the national blood programs' achievements and challenges and share best practices and lessons learned.

e To highlight the continuous need for regular, unpaid blood donation to achieve universal access to safe blood transfusion.

e To promote a culture of regular blood donation among the Youth and the general public and to increase the diversity and sustainability of the blood donor pool.

The suggested action plan for the event is as follows:
e The event will aim to maximize the registration of willing blood donors along with optimal blood collection.

To spread awareness among rural masses, activities of felicitation of blood donors and pledge-taking ceremonies may be organized at the Gram Panchayat level. The respective Sarpanch can administer the pledge.

o Similar, activities may be organized at the blood and district levels. The office of the District Collector/District Magistrate may supervise all activities in the concerned district.

e A pledge in English or Hindi or regional language for blood donation will be administered through both physical events at the levels and digital platforms. A template of the Pledge in English and Hindi is enclosed, which may be translated into regional languages for wider dissemination.

.

To promote the registration of blood donors and create awareness, a facility for blood group testing may be made available on 1st October 2024 at all PHCs/CHCs/Sub-district and District Hospitals and other blood centres as per feasibility. This will help the common citizens to know their blood group, which will be useful in emergencies for blood donation.

o All Central Government Ministries, hospitals/Institutions, and Medical Colleges 3.

(Government and Private) are requested to organize similar activities on 1st October 2024.

Accordingly, you are requested to organize activities to celebrate the event and 4.

disseminate the information widely among various stakeholders, Further, it is requested, that a report along with the photographs of the 5.

celebration of the event may be shared with this ministry to Dr. Megha Pravin Khobragade, ADG, on the email address: mp.khobragade@gov.in Yours sincerely.

(Apurva Chandra)
Encl : as above 1)  Secretary to Govt of India, All Ministries/Departments 2)  Additional Chief Secretary/Principal Secretary/Secretary (Health & FW)
AII States/UTs.

ff
( )NBTC
State Logo NATONAL VOLUNTARY
n es BY ATTON DAY
-), FOG (0 ten enging 20 years or dyna rity of the political Be Different: Donate Blood , Donate Life
organised by State Name

# शप राष्टीय  स्वैच्छिक_रक्तदान_दिवस, 2024

आज,मैं,.

..

2024 के दिन यह शपथ लेता/लेती हूँ : कि मैं अपना रक्त नियमित रूप से दान करूँगा/करुँगीI
भारत में रक्त की विशाल आवश्यकता को ध्यान में रखते हए, मैं यह भी वचन देता/देती हँ कि मैं अपने परिवार, दोस्तों, रिश्तेदारों, सहयोगियों और जनता को नियमित, स्वैच्छिक अवैतनिक रक्तदान की आवश्यकता के बारे में जागरूक करने का प्रयास करूँगा/करुँगीI
इसके साथ-साथ मैं यह भी वचन देता/देती हँ कि जब भी कभी, किसी को रक्त की जरुरत होगी, मैं अपने खर्चे पर, बिना किसी लोभ-लालच के, जाति-धर्म के भेदभाव से मुक्त होकर रक्तदान करूँगा/करूँगी।
में सतत प्रयास करूँगा/करूँगी ताकि रक्त की कमी से हमारे आस-पास किसी
की जान न जाएा

# Pledge National Voluntary Blood Donation Day, 2024

| I .                                                                   | am,        | the content of the conte   | resident   |
|-----------------------------------------------------------------------|------------|----------------------------|------------|
| of_                                                                   |            |                            |            |
| (Village)                                                             | (District) | (States),                  |            |
| today, on ist October, National Voluntary Blood Donation Day, 2024 do |            |                            |            |
| hereby pledge to donate my blood regularly.                           |            |                            |            |

hereby pledge to donate my blood regularly. Keeping in view the huge need for blood in India. I also undertake to create awareness among my family members, friends, relatives, colleagues, and the public about the need for regular, voluntary unpaid blood donation. Along with this, I also undertake that whenever someone needs blood, I shall donate blood at my own expense without any greed, free from castereligion discrimination.

I will make relentless efforts so that no life is lost around us due to a shortage of blood.

## Streaming Support

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
data = SimpleDirectoryReader(
    input_dir="./drive/MyDrive/ccs_computer/icmr_parsed",
    required_exts=['.md'],
    recursive=True,
).load_data()
index = VectorStoreIndex.from_documents(data)

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="context", llm=llm)

In [ ]:
response = chat_engine.stream_chat("what is निदेशक/प्रभारी  निदेशक परिषद के सभी?")
for token in response.response_gen:
    print(token, end="")

The phrase "निदेशक/प्रभारी निदेशक परिषद के सभी" translates to "Director/Deputy Director of all the Councils" in English. If you have any specific questions or need more information related to this topic, feel free to ask!